<table>
    <tr>
        <td><img src="./img/Macc.png" width="auto"/></td>
        <td>
            <table><tr>
            <h1 style="color:blue;text-align:center">Lógica para Ciencias de la Computación</h1></td>
            </tr></table>   
        <td>&nbsp;</td>
        <td>
            <table><tr>
            <tp><p style="font-size:150%;text-align:center">Taller</p></tp>
            <tp><p style="font-size:150%;text-align:center">Algoritmo WalkSAT</p></tp>
            </tr></table>
        </td>
    </tr>
</table>

---

# Objetivo <a class="anchor" id="inicio"></a>

En clase hemos visto el algoritmo walkSAT para encontrar un modelo $I$ para una fórmula $S$ en forma clausal. En este notebook implementaremos este algoritmo en Python.

Desarrollaremos la explicación mediante las siguientes secciones.

# Secciones

1. [Funciones auxiliares](#aux)
2. [Walksat](#walk)
3. [Comparación de tiempos](#comp)


# Funciones auxiliares <a class="anchor" id="aux"></a>

([Volver al inicio](#inicio))

Para poder implementar el algoritmo walkSAT requerimos algunas funciones auxiliares:

* `complemento()`: Sea $\ell$ un literal. Se define 

`complemento`$(\ell)=\begin{cases}
\overline{p},&\mbox { si }\ell=p\mbox{ para algún }p\\ 
p,&\mbox{ si }\ell=\overline{p}\mbox{ para algún }p\\
\end{cases}$

* `interpretacion_aleatoria(letrasp)`: Devuelve una interpretación seleccionando aleatoriamente el valor True o False para cada letra proposicional en `letrasp`. 

* `flip_literal`$(I, \ell)$: Modifica el valor de verdad asignado por la interpretación $I$ al literal $\ell$.

In [1]:
from random import *
from copy import deepcopy

def complemento(l):
    if '-' in l:
        return l[1:]
    else:
        return '-' + l

def interpretacion_aleatoria(letrasp):
    I = {p:randint(0,1)==1 for p in letrasp}
    return I

def flip_literal(I, l):
    p = l[-1]
    valor = False if I[p] else True
    Ip = deepcopy(I)
    Ip[p] = valor
    return Ip

In [2]:
letrasp = ['p', 'q', 'r']
I = interpretacion_aleatoria(letrasp)
I

{'p': True, 'q': True, 'r': True}

In [3]:
I = flip_literal(I, 'p')
I

{'p': False, 'q': True, 'r': True}

Para manejar la información sobre las letras proposicionales y las cláusulas de la fórmula que vayamos a trabajar, hemos definido una clase `WalkSatEstado` de la siguiente manera:

In [4]:
class WalkSatEstado():
    
    def __init__(self, S):
        self.S = S
        self.letrasp = list(set([l[-1] for C in self.S for l in C]))
        self.I = interpretacion_aleatoria(self.letrasp)
        self.I_lits = set([p for p in self.letrasp if self.I[p]] + ['-'+p for p in self.letrasp if not self.I[p]])
        self.clausulas_sat = [C for C in self.S if any((True for x in self.I_lits if x in C))]
        self.clausulas_unsat = [C for C in self.S if C not in self.clausulas_sat]

    def actualizar(self, I):
        self.I = I
        self.I_lits = set([p for p in self.letrasp if self.I[p]] + ['-'+p for p in self.letrasp if not self.I[p]])
        self.clausulas_sat = [C for C in self.S if any((True for x in self.I_lits if x in C))]
        self.clausulas_unsat = [C for C in self.S if C not in self.clausulas_sat]
       
    def SAT(self):
        return len(self.clausulas_unsat) == 0

    def break_count(self, l):
        if l in self.I_lits:
            lit = l
        else:
            lit = complemento(l)
        clausulas_break_count = [C for C in self.clausulas_sat if set(C).intersection(self.I_lits)=={lit}]
        return len(clausulas_break_count)

Consideremos una fórmula en forma clausal y veamos el correspondiente objeto `WalkSatEstado`:

In [5]:
S = [['r'], ['p', '-r'], ['q', '-r'], ['-p', '-q', 'r']]
w = WalkSatEstado(S)
print("Fórmula:", S)
print("Letras proposicionales:", w.letrasp)
print("Interpretación:", w.I)
print("Literales determinados por la interpretación:", w.I_lits)
print("Cláusulas sat:", w.clausulas_sat)
print("Cláusulas unsat", w.clausulas_unsat)
print("I hace verdadera la fórmula?:", w.SAT())

Fórmula: [['r'], ['p', '-r'], ['q', '-r'], ['-p', '-q', 'r']]
Letras proposicionales: ['q', 'r', 'p']
Interpretación: {'q': False, 'r': False, 'p': True}
Literales determinados por la interpretación: {'-q', '-r', 'p'}
Cláusulas sat: [['p', '-r'], ['q', '-r'], ['-p', '-q', 'r']]
Cláusulas unsat [['r']]
I hace verdadera la fórmula?: False


Observe que esta clase tiene definido el método `break_count()` para determinar cuántas cláusulas sat se vuelven unsat si se voltea el valor de un literal:

In [6]:
w.break_count('-r')

1

---

# WalkSAT <a class="anchor" id="walk"></a>

([Volver al inicio](#inicio))

Presentamos ahora el algoritmo walkSAT.

<img src="./img/walksat.png" width="auto"/>

**Ejercicio 1:**

Implemente la función `walkSAT`.

Pruebe su respuesta con los siguientes ejemplos:

1. `S = [['r'], ['p', '-r'], ['q', '-r'], ['-p', '-q', 'r']]` 

Solución:  $I\,{=}\,\{r: True, p: True, q: True\}$.

2. `S=[['s'], ['-r', '-q'], ['r', 'q'], ['p', '-s'], ['r', '-s'], ['-p', '-r', 's']]` 

Solución: $I\,{=}\,\{s: True, p: True, r: True, q: False\}$.

3. `S = [['s'], ['p','-t'], ['q','-t'], ['-p','-q','t'], ['t','-s'], ['r','-s'], ['-t','-r','s']]` 

Solución: $I\,{=}\,\{s: True, p: True, r: True, q: False\}$.

4. `S = [['p','-q'],['-p','-q'],['q','r'],['-q''-r'],['-p','-r'],['p','-r']]` 

Solución: Intento fallido.

In [11]:
def walkSAT(A, max_flips=10, max_tries=10, p=0.2):
    w = WalkSatEstado(A)
    for i in range(max_tries):
        w.actualizar(interpretacion_aleatoria(w.letrasp))
        for j in range(max_flips):
            if w.SAT():
                return "Satisfacible", w.I
            C = choice(w.clausulas_unsat)
            breaks = [(l, w.break_count(l)) for l in C]
            if any((x[1] == 0 for x in breaks)) > 0:
                v = choice(breaks)[0]
            else:
                if uniform(0, 1) < p:
                    assert (len(C) > 0), f"{C}"
                    v = choice(C)
                else:
                    min_count = min([x[1] for x in breaks])
                    mins = [x[0] for x in breaks if x[1] == min_count]
                    v = choice(mins)
            I = flip_literal(w.I, v)
            w.actualizar(I)
    print("Intento Fallido")
    return None


S = [['r'], ['p', '-r'], ['q', '-r'], ['-p', '-q', 'r']]
print("Fórmula:", S)
print(walkSAT(S))


Fórmula: [['r'], ['p', '-r'], ['q', '-r'], ['-p', '-q', 'r']]
('Satisfacible', {'q': True, 'r': True, 'p': True})


---

# Comparación de tiempos <a class="anchor" id="comp"></a>

([Volver al inicio](#inicio))

Comparamos los tiempos de búsqueda de modelo para fórmulas aleatorias cada vez más grandes, entre los algoritmos DPLL y walkSAT.

**Ejercicio 2:**

Incluya aquí su implementación del DPLL.

In [ ]:
from Logica import *

In [14]:
from time import time
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

def obtiene_tiempos(fun, args, num_it=100):
    tiempos_fun = []
    for i in range(num_it):
        arranca = time()
        x = fun(*args)
        para = time()
        tiempos_fun.append(para - arranca)
    return tiempos_fun

def compara_entradas_funs(funs, nombres_funs, lista_args, N=100):
    entradas = []
    funcion = []
    tiempos = []
    lista_dfs = []
    for i, args in enumerate(lista_args):
        for j, fun in enumerate(funs):
            t = obtiene_tiempos(fun, [args], N)
            tiempos += t
            n = len(t)
            entradas += [i+1]*n
            funcion += [nombres_funs[j]]*n
        df = pd.DataFrame({'Long_entrada':entradas, 
                           'Funcion':funcion,
                           'Tiempo_prom':tiempos})
        lista_dfs.append(df)
    df = pd.concat(lista_dfs).reset_index()
    sns.lineplot(x='Long_entrada',y='Tiempo_prom',hue='Funcion',data=df)
    plt.show()

In [15]:
from random import uniform, choice

cantidad = 40
letras = [chr(i) for i in range(257, 257+cantidad)]
lista = [letras[0]]
formula = letras[0]
for p in letras[1:]:
            neg1 = '-' if uniform(0,1) < .5 else ''
            neg2 = '-' if uniform(0,1) < .5 else ''
            neg3 = '-' if uniform(0,1) < .5 else ''
            conectivo = choice(['Y','O','>'])
            if uniform(0,1) < .5:
                formula = neg1 + "(" + neg2 + formula + conectivo + neg3 + p + ")"
            else:
                formula = neg1 + "(" + neg2 + p + conectivo + neg3 + formula + ")"
            lista.append(formula)

In [20]:
nombres = ['dpll', 'walksat']
sat_dpll = lambda formula: dpll(tseitin(formula), {})
sat_walksat = lambda formula: walkSAT(tseitin(formula))
funs = [sat_dpll, sat_walksat]
compara_entradas_funs(funs, nombres, lista)

NameError: name 'tseitin' is not defined

Vemos que en todos los casos el DPLL es más rápido que el walkSAT.

---